In [1]:
import pandas as pd
import numpy as np
import time
from datetime import datetime

In [2]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

def load_df(csv_path='../input/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'},
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

print(os.listdir("../input"))

['sample_submission.csv', 'train_v2.csv', 'test_v2.csv', 'sample_submission_v2.csv', 'train.csv', 'test.csv']


In [3]:
%%time
train_df = load_df()

Loaded train.csv. Shape: (903653, 55)
CPU times: user 2min 50s, sys: 9.64 s, total: 2min 59s
Wall time: 3min 15s


In [4]:
train_df.shape

(903653, 55)

In [5]:
train_df.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,device.browserVersion,device.deviceCategory,device.flashVersion,device.isMobile,device.language,device.mobileDeviceBranding,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.mobileDeviceModel,device.mobileInputSelector,device.operatingSystem,device.operatingSystemVersion,device.screenColors,device.screenResolution,geoNetwork.city,geoNetwork.cityId,geoNetwork.continent,geoNetwork.country,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.networkLocation,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.transactionRevenue,totals.visits,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,not available in demo dataset,not available in demo dataset,Izmir,not available in demo dataset,Asia,Turkey,not available in demo dataset,not available in demo dataset,(not set),ttnet.com.tr,not available in demo dataset,Izmir,Western Asia,1,1,1,1,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Oceania,Australia,not available in demo dataset,not available in demo dataset,not available in demo dataset,dodo.net.au,not available in demo dataset,not available in demo dataset,Australasia,1,1,1,1,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,not available in demo dataset,not available in demo dataset,Madrid,not available in demo dataset,Europe,Spain,not available in demo dataset,not available in demo dataset,(not set),unknown.unknown,not available in demo dataset,Community of Madrid,Southern Europe,1,1,1,1,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not a

In [6]:
target = train_df['totals.transactionRevenue'].fillna(0).astype(float)

> # remove 0 variance columns

In [7]:
train_df.nunique()[train_df.nunique() == 1]

socialEngagementType                                 1
device.browserSize                                   1
device.browserVersion                                1
device.flashVersion                                  1
device.language                                      1
device.mobileDeviceBranding                          1
device.mobileDeviceInfo                              1
device.mobileDeviceMarketingName                     1
device.mobileDeviceModel                             1
device.mobileInputSelector                           1
device.operatingSystemVersion                        1
device.screenColors                                  1
device.screenResolution                              1
geoNetwork.cityId                                    1
geoNetwork.latitude                                  1
geoNetwork.longitude                                 1
geoNetwork.networkLocation                           1
totals.bounces                                       1
totals.new

In [8]:
train_df.drop(['device.browserSize'
,'device.browserVersion'
,'device.flashVersion'
,'device.language'
,'device.mobileDeviceBranding'
,'device.mobileDeviceInfo'
,'device.mobileDeviceMarketingName'
,'device.mobileDeviceModel'
,'device.mobileInputSelector'
,'device.operatingSystemVersion'
,'device.screenColors'
,'device.screenResolution'
,'geoNetwork.cityId'
,'geoNetwork.latitude'
,'geoNetwork.longitude'
,'geoNetwork.networkLocation'
,'trafficSource.adwordsClickInfo.criteriaParameters'],axis=1,inplace=True)

In [9]:
train_df.shape

(903653, 38)

In [10]:
train_df.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,geoNetwork.city,geoNetwork.continent,geoNetwork.country,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.transactionRevenue,totals.visits,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,desktop,False,Windows,Izmir,Asia,Turkey,(not set),ttnet.com.tr,Izmir,Western Asia,1,1,1,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,desktop,False,Macintosh,not available in demo dataset,Oceania,Australia,not available in demo dataset,dodo.net.au,not available in demo dataset,Australasia,1,1,1,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,desktop,False,Windows,Madrid,Europe,Spain,(not set),unknown.unknown,Community of Madrid,Southern Europe,1,1,1,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,desktop,False,Linux,not available in demo dataset,Asia,Indonesia,not available in demo dataset,unknown.unknown,not available in demo dataset,Southeast Asia,1,1,1,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,NaN,google + online,organic,NaN,google
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,mobile,True,Android,not available in demo dataset,Europe,United Kingdom,not available in demo dataset,unknown.unknown,not available in demo dataset,Northern Europe,1,1,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,True,(not provided),organic,NaN,google


In [11]:
train_df['totals.bounces'] = train_df['totals.bounces'].fillna('0')
train_df['totals.newVisits'] = train_df['totals.newVisits'].fillna('0')
train_df['trafficSource.adwordsClickInfo.isVideoAd'] = train_df['trafficSource.adwordsClickInfo.isVideoAd'].fillna(True)
train_df['trafficSource.isTrueDirect'] = train_df['trafficSource.isTrueDirect'].fillna(False)

In [12]:
columns = [col for col in train_df.columns if train_df[col].nunique() > 1]


In [13]:
columns

['channelGrouping',
 'date',
 'fullVisitorId',
 'sessionId',
 'visitId',
 'visitNumber',
 'visitStartTime',
 'device.browser',
 'device.deviceCategory',
 'device.isMobile',
 'device.operatingSystem',
 'geoNetwork.city',
 'geoNetwork.continent',
 'geoNetwork.country',
 'geoNetwork.metro',
 'geoNetwork.networkDomain',
 'geoNetwork.region',
 'geoNetwork.subContinent',
 'totals.bounces',
 'totals.hits',
 'totals.newVisits',
 'totals.pageviews',
 'totals.transactionRevenue',
 'trafficSource.adContent',
 'trafficSource.adwordsClickInfo.adNetworkType',
 'trafficSource.adwordsClickInfo.gclId',
 'trafficSource.adwordsClickInfo.isVideoAd',
 'trafficSource.adwordsClickInfo.page',
 'trafficSource.adwordsClickInfo.slot',
 'trafficSource.campaign',
 'trafficSource.isTrueDirect',
 'trafficSource.keyword',
 'trafficSource.medium',
 'trafficSource.referralPath',
 'trafficSource.source']

In [14]:
train_df = train_df[columns]


## Feature Engineering

In [15]:
train_df['diff_visitId_time'] = train_df['visitId'] - train_df['visitStartTime']
train_df['diff_visitId_time'] = (train_df['diff_visitId_time'] != 0).astype(int)
del train_df['visitId']
del train_df['sessionId']

In [16]:
format_str = '%Y%m%d' 
train_df['formated_date'] = train_df['date'].apply(lambda x: datetime.strptime(str(x), format_str))
train_df['month'] = train_df['formated_date'].apply(lambda x:x.month)
train_df['quarter_month'] = train_df['formated_date'].apply(lambda x:x.day//8)
train_df['day'] = train_df['formated_date'].apply(lambda x:x.day)
train_df['weekday'] = train_df['formated_date'].apply(lambda x:x.weekday())

del train_df['date']
del train_df['formated_date']

In [17]:
train_df['totals.hits'] = train_df['totals.hits'].astype(int)
train_df['mean_hits_per_day'] = train_df.groupby(['day'])['totals.hits'].transform('mean')
del  train_df['day']

In [18]:
train_df['formated_visitStartTime'] = train_df['visitStartTime'].apply(
    lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x)))
train_df['formated_visitStartTime'] = pd.to_datetime(train_df['formated_visitStartTime'])
train_df['visit_hour'] = train_df['formated_visitStartTime'].apply(lambda x: x.hour)

del train_df['visitStartTime']
del train_df['formated_visitStartTime']

## label encoding

In [19]:
for col in train_df.columns:
    if col in ['fullVisitorId', 'month', 'quarter_month', 'weekday', 'visit_hour', 'WoY']: continue
    if train_df[col].dtypes == object or train_df[col].dtypes == bool:
        train_df[col], indexer = pd.factorize(train_df[col])

In [20]:
numerics = [col for col in train_df.columns if 'totals.' in col]
numerics += ['visitNumber', 'mean_hits_per_day', 'fullVisitorId']
categorical_feats =  [col for col in train_df.columns if col not in numerics]

In [21]:
for col in categorical_feats:
    train_df[col] = train_df[col].astype(int)


In [22]:
train_df.fullVisitorId

0         1131660440785968503
1          377306020877927890
2         3895546263509774583
3         4763447161404445595
4           27294437909732085
5         2938943183656635653
6         1905672039242460897
7          537222803633850821
8         4445454811831400414
9         9499785259412240342
10        0523069750702990437
11         982320996976275749
12         357659889600827884
13        1438082600262726746
14        3531015320757710684
15        9638207207743070032
16        9876750586615598787
17        2222266935962032743
18        9674781571160116268
19        3696906537737368442
20        4478318070775453050
21        6098154234696452861
22        3323434834508685818
23        3053576296023059465
24         702736826487255366
25        8794587387581803040
26        3293772317640873624
27        1283542838194038522
28        4339756682310369249
29        7598520899202646361
                 ...         
903623    6636384798982309878
903624    3949036270578731944
903625    

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_df.drop(['fullVisitorId'],axis=1), target, test_size=0.33, random_state=42)

In [24]:
X_train.head()

,channelGrouping,visitNumber,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,geoNetwork.city,geoNetwork.continent,geoNetwork.country,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.transactionRevenue,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,diff_visitId_time,month,quarter_month,weekday,mean_hits_per_day,visit_hour
756972,4,1,0,1,1,3,1,3,33,1,114,1,10,0,1,0,0,-1,-1,-1,-1,0,-1,-1,0,1,-1,4,-1,8,0,4,0,6,4.638119,3
899004,0,1,0,1,1,3,1,3,31,1,143,1,12,1,8,0,7,-1,-1,-1,-1,0,-1,-1,0,0,0,0,-1,0,0,5,3,2,4.569723,0
870223,0,1,4,1,1,4,1,2,47,1,123,1,4,0,1,0,0,-1,-1,-1,-1,0,-1,-1,0,0,0,0,-1,0,0,1,0,5,4.680462,4
335571,6,1,0,0,0,0,1,0,44,1,2,1,3,0,1,0,0,-1,-1,-1,-1,0,-1,-1,0,0,-1,1,69,25,0,11,0,4,4.449207,13
631586,0,1,0,1,1,3,11,0,9,0,2,10,5,0,1,0,0,-1,-1,-1,-1,0,-1,-1,0,0,0,0,-1,0,0,5,1,5,4.772247,14


In [25]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

rf.fit(X_train,y_train)

y_pred = rf.predict(X_test)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [26]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(y_test, y_pred))

In [27]:
rms

48674533.39341315

In [28]:
#Done